# 2. FERPlus XGBoost 분류

In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

## 2.1. 학습에 사용될 하이퍼 패러미터 선언 및 Random Seed 설정

In [2]:
np.random.seed(201)

In [3]:
FERPLUS_DIR = "datasets/ferPlus/prep/"
INPUT_TRAIN_DATA = 'ferPlus_X.npy'
LABEL_TRAIN_DATA = 'ferPlus_y.npy'

TEST_SPLIT = 0.2
RND_SEED = 0

In [4]:
input_data = np.load(open(FERPLUS_DIR + INPUT_TRAIN_DATA, 'rb'))
label_data = np.load(open(FERPLUS_DIR + LABEL_TRAIN_DATA, 'rb'))

In [5]:
nsamples, nx, ny, nc = input_data.shape
input_data = input_data.reshape((nsamples,nx*ny*nc))

In [6]:
input_data.shape

(13726, 2304)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(input_data,
                                                    label_data,
                                                    test_size=TEST_SPLIT,
                                                    random_state=RND_SEED)

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_train,
                                                  y_train,
                                                  test_size=TEST_SPLIT,
                                                  random_state=RND_SEED)

In [9]:
print(x_train.shape)
print(x_val.shape)
print(x_test.shape)

(8784, 2304)
(2196, 2304)
(2746, 2304)


## 2.2 XGBoost 모델을 이용하여 학습

In [10]:
clf = XGBClassifier(n_estimators=1000, booster='gbtree',
                    tree_method ='gpu_hist',
                    predictor = 'gpu_predictor')
clf.fit(x_train, np.argmax(y_train, axis=1))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', predictor='gpu_predictor',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='gpu_hist', validate_parameters=False,
              verbosity=None)

검증셋 정확도

In [11]:
clf.score(x_val, np.argmax(y_val, axis=1))

0.8073770491803278

테스트셋 정확도

In [12]:
clf.score(x_test, np.argmax(y_test, axis=1))

0.8055353241077932

## 2.3 K-fold  Cross Validation 이용

In [13]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=10, random_state=RND_SEED, shuffle=True)

for train_index, test_index in kf.split(input_data):
    X_train, X_test = input_data[train_index], input_data[test_index]
    Y_train, Y_test = label_data[train_index], label_data[test_index]

In [14]:
clf = XGBClassifier(n_estimators=1000, booster='gbtree',
                    tree_method ='gpu_hist',
                    predictor = 'gpu_predictor')
clf.fit(X_train, np.argmax(Y_train, axis=1))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=0,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', predictor='gpu_predictor',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              subsample=1, tree_method='gpu_hist', validate_parameters=False,
              verbosity=None)

테스트셋 정확도

In [15]:
clf.score(X_test, np.argmax(Y_test, axis=1))

0.8090379008746356